<h1>
<center>Comparing the Performance of popular ML algorithms</center>
</h1>

<center><font size="3">
In this notebook we structure several of the common machine learning algorithms to compare their results with our approach.
<br>
<br>
Αiming to make as fair a comparison as possible, we perform hyper-parameter optimization οn every algorithm using the validation set. 
<br>
<br>
Then knowing the best combination of hyper-parameters we train each model and evaluate the model on test set by measuring various metrics

</font></center>

## Generals

<font size="3"> 
Packages import and system configurations. 
</font>

In [ ]:
#Data
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import shutil
import math
from datetime import datetime
from datetime import datetime as dt
import itertools
#Graph Counstruction
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor 
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.linear_model import LinearRegression 
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF
from sklearn import metrics
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

<font size="3"> 
Define necessary paths. 
</font>

In [ ]:
train_path = 'io/input/base_data/train.csv'
test_path = 'io/input/base_data/test.csv'
val_path ='io/input/base_data/valid.csv'
plot_export_path = 'io/output/exports/analysis_plots/'

## Core Functionality

<font size="3"> 
Load data according to validation_ind (if true: return train-validation | else: return train - test) 
</font>

In [ ]:
def load_data(train_path,val_path,test_path,validation_ind):
    if validation_ind:
        trainset = pd.read_csv(train_path,index_col=0)
        x_train,y_train = init_process(trainset) 
        valset = pd.read_csv(val_path,index_col=0)
        x_test,y_test = init_process(valset)
        x_train,x_test = scaller(x_train,x_test)
    else:
        trainset = pd.read_csv(train_path,index_col=0)
        valset = pd.read_csv(val_path,index_col=0)
        trainset = pd.concat([trainset, valset], ignore_index=True)
        x_train,y_train = init_process(trainset) 
        testset = pd.read_csv(test_path,index_col=0)
        x_test,y_test = init_process(testset)
        x_train,x_test = scaller(x_train,x_test)
    return x_train,y_train,x_test,y_test

<font size="3"> 
Split: features -  target
</font>

In [ ]:
def init_process(df):
    y = df['Target']
    x = df.drop(['Target'], axis=1)
    x = x.drop(['Asset_id'], axis=1)
    return x, y

<font size="3"> 
Apply Standardization. 
</font>

In [ ]:
def scaller(x_train,x_test):
    Standar_Scaller = StandardScaler()
    scalled_train_data= Standar_Scaller.fit_transform(x_train)
    scalled_test_data = Standar_Scaller.transform(x_test)
    return scalled_train_data,scalled_test_data

## Hyper-Parameters Tunig

<font size="3">
A core function that apply the evaluation procces using train & validation set:
<ol>
<li>Initialize the given regression models (SVR, DecisionTreeRegressor, XGBRegressor & more) with default hyperparameters.</li>
<li>Define the hyperparameter search spaces (ranges of values to try for each hyperparameter) for each model.</li>
<li>Loop through the three models and their corresponding hyperparameter search spaces.</li>
<li>Generate all possible combinations of hyperparameters for each model.</li>
<li>For each hyperparameter combination, fit the model on the training set, make predictions on the validation set, and calculate the MAE score.</li>
<li>Track the best hyperparameters for each model based on the lowest MAE score on the validation set.</li>
<li>Return the best hyperparameters for each model as a list of dictionaries.</li>

In [ ]:
def parameter_tuning_validation(x_train, y_train, x_val, y_val):
    # Initialize the models
    svm_reg = SVR()
    tree_reg = DecisionTreeRegressor()
    xgb_reg = XGBRegressor()
    nb_reg = GaussianProcessRegressor()
    lr_reg = LinearRegression()
    knn_reg = KNeighborsRegressor()
    rf_reg = RandomForestRegressor()
    gb_reg = GradientBoostingRegressor()
    ada_reg = AdaBoostRegressor()

    # Define the hyperparameter ranges
    svm_param_grid = {'kernel': ['linear','sigmoid', 'rbf'],'C': [0.1, 10, 15],'gamma':['scale','auto']}
    tree_param_grid = {'criterion':['squared_error','friedman_mse'],'max_depth': [6,10,12], 'min_samples_split': [4, 6, 8]}
    xgb_param_grid = {'n_estimators': [6, 10, 25], 'max_depth': [1, 2, 5], 'eta':[0.1,0.3,0.5]}
    nb_param_grid = {'kernel': [1**2 * RBF(length_scale=1)],'alpha': [0.1, 1.0, 10.0],'n_restarts_optimizer': [0, 1, 2],'optimizer': ['fmin_l_bfgs_b']}
    lr_param_grid = {'fit_intercept': [True, False], 'normalize': [True, False]}
    knn_param_grid = {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance'], 'algorithm': ['ball_tree', 'kd_tree', 'brute']}
    rf_param_grid = {'max_depth': [3, 5, 7], 'n_estimators': [50, 100, 200], 'min_samples_split': [2, 4, 6]}
    gb_param_grid = {'learning_rate': [0.01, 0.1, 1.0], 'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7]}
    ada_param_grid = {'learning_rate': [0.01, 0.1, 1.0], 'n_estimators': [50, 100, 200]}

    
    models = [svm_reg, tree_reg, xgb_reg, nb_reg, lr_reg, knn_reg, rf_reg, gb_reg, ada_reg]
    param_grids = [svm_param_grid, tree_param_grid, xgb_param_grid, nb_param_grid, lr_param_grid,
                  knn_param_grid, rf_param_grid, gb_param_grid, ada_param_grid]

    best_params = []
    for i, model in enumerate(models):
        print('\n')
        best_mae = float('inf')
        best_mape = float('inf')
        best_params_i = {}
        # Generate all possible combinations of hyperparameters
        param_combinations = list(itertools.product(*(param_grids[i][param] for param in param_grids[i])))
        for j, params in enumerate(param_combinations):
            # Unpack the tuple of parameter values into individual arguments
            params_dict = dict(zip(param_grids[i], params))
            model.set_params(**params_dict)
            model.fit(x_train, y_train)
            y_val_pred = model.predict(x_val)
            mse = round(mean_squared_error(y_val, y_val_pred), 7)
            mae = round(mean_absolute_error(y_val, y_val_pred),7)
            r2 = round(r2_score(y_val, y_val_pred),7)
            rmse = round(np.sqrt(mean_squared_error(y_val, y_val_pred)), 7)
            param_str = ', '.join([f'{param}={value}' for param, value in params_dict.items()])
            print(f"Experiment {j+1} with {model.__class__.__name__} using {param_str} has MSE: {mse}, MAE: {mae}, R2: {r2}, RMSE: {rmse}")
            if mae < best_mae:
                best_mae = mae
                best_r2 = r2
                best_params_i = dict(zip(param_grids[i], params))
        best_params.append(best_params_i)
    return best_params

## Model Train & Test Evalaution 

<font size="3">
A core function that apply the final train and evaluation procces using train & test set:
<ol>
<li>Initialize the models with the best hyperparameters.</li>
<li>Fit each model on the training data.</li>
<li>Use the fitted models to make predictions on the test set.</li>
<li>Compute the evaluation metrics (MSE, MAE, R2, RMSE) for each model.</li>
<li>Store the results for each model in a dictionary.</li>
<li>Print the evaluation metrics for each model.</li>
<li>Return the dictionary containing the results.</li>

In [ ]:
def evaluate_models(x_train, y_train, x_test, y_test, best_params):
    # Initialize the models with the best hyperparameters
    svm_reg = SVR(**best_params[0])
    tree_reg = DecisionTreeRegressor(**best_params[1])
    xgb_reg = XGBRegressor(**best_params[2])
    nb_reg = GaussianProcessRegressor(**best_params[3])
    lr_reg = LinearRegression(**best_params[4])
    knn_reg = KNeighborsRegressor(**best_params[5])
    rf_reg = RandomForestRegressor(**best_params[6])
    gb_reg = GradientBoostingRegressor(**best_params[7])
    ada_reg = AdaBoostRegressor(**best_params[8])

    models = [svm_reg, tree_reg, xgb_reg, nb_reg, lr_reg, knn_reg, rf_reg, gb_reg, ada_reg]
    model_names = ['SVM', 'Decision Tree', 'XGBoost', 'Gaussian NB', 'Linear Regression',
                  'KNN', 'Random Forest', 'Gradient Boosting','AdaBoost']
    
    results = {}
    for i, model in enumerate(models):
        model.fit(x_train, y_train)
        y_test_pred = model.predict(x_test)
        mse = round(mean_squared_error(y_test, y_test_pred), 7)
        mae = round(mean_absolute_error(y_test, y_test_pred),7)
        r2 = round(r2_score(y_test, y_test_pred),7)
        rmse = round(np.sqrt(mean_squared_error(y_test, y_test_pred)), 7)
        results[model_names[i]] = {'MSE': mse,'MAE': mae, 'R2': r2, 'RMSE': rmse}
        print(f"{model_names[i]} model has MSE: {mse}, MAE: {mae}, R2: {r2}, RMSE: {rmse}")
    return results

<font size="3"> 
Plot metrics on subplots for camparison purposes
</font>

In [ ]:
def plot_all_metrics_train_val(metrics):
    model_names = list(results.keys())
    mse = [results[model]['MSE'] for model in model_names]
    mae = [results[model]['MAE'] for model in model_names]
    r2 = [results[model]['R2'] for model in model_names]
    rmse = [results[model]['RMSE'] for model in model_names]
    
    x = np.arange(len(model_names))
    width = 0.35
    
    fig, axs = plt.subplots(nrows=2, ncols=2,figsize=(12, 8))
    fig.suptitle('Comparison of Model Performance', fontsize=16)
    # Plot the first metric on the top-left subplot
    axs[0, 0].bar(x - width/2, mse, width, label='MSE')
    axs[0, 0].set_ylabel('MSE')
    axs[0, 0].set_title('MSE')
    axs[0, 0].set_xticks(x)
    axs[0, 0].set_xticklabels(model_names)
    #axs[0, 0].set_ylim(0,1.2)
    axs[0, 0].legend()
    # Plot the second metric on the top-right subplot
    axs[0, 1].bar(x - width/2, mae, width, label='MAE')
    axs[0, 1].set_ylabel('MAE')
    axs[0, 1].set_title('MAE')
    axs[0, 1].set_xticks(x)
    axs[0, 1].set_xticklabels(model_names)
    #axs[0, 1].set_ylim(0,1.2)
    axs[0, 1].legend()
    # Plot the third metric on the bottom-left subplot
    axs[1, 0].bar(x - width/2, r2, width, label='R2')
    axs[1, 0].set_ylabel('R2')
    axs[1, 0].set_title('R2')
    axs[1, 0].set_xticks(x)
    axs[1, 0].set_xticklabels(model_names)
    #axs[1, 0].set_ylim(0,1.2)
    axs[1, 0].legend()
    # Plot the fourth metric on the bottom-right subplot
    axs[1, 1].bar(x - width/2, rmse, width, label='RMSE')
    axs[1, 1].set_ylabel('RMSE')
    axs[1, 1].set_title('RMSE')
    axs[1, 1].set_xticks(x)
    axs[1, 1].set_xticklabels(model_names)
    #axs[1, 1].set_ylim(0,1.2)
    axs[1, 1].legend()
    # Adjust the spacing between subplots
    plt.tight_layout()
    # Show the plot
    plt.show()

## Pipeline Execution

<font size="3"> 
Hyper-Parameter tuning
</font>

In [ ]:
x_train,y_train,x_val,y_val = load_data(train_path,val_path,test_path,validation_ind=True)
best_params = parameter_tuning_validation(x_train, y_train, x_val, y_val)

<font size="3"> 
Training and evaluation using testset
</font>

In [ ]:
# best_params=[{'kernel': 'linear', 'C': 0.1, 'gamma': 'scale'},
#  {'criterion': 'squared_error', 'max_depth': 6, 'min_samples_split': 4},
#  {'n_estimators': 25, 'max_depth': 1, 'eta': 0.3}]

In [ ]:
x_train,y_train,x_test,y_test = load_data(train_path,val_path,test_path,validation_ind=False)
results =  evaluate_models(x_train, y_train, x_test, y_test, best_params)

<font size="3"> 
Plot results for comparison purposes 
</font>

In [ ]:
plot_all_metrics_train_val(results)